In [1]:
import pathlib

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [3]:
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import pytesseract
import tensorflow as tf
import re

print(tf.__version__)

2.1.0


In [4]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [5]:
tf.gfile = tf.io.gfile

utils_ops.tf = tf.compat.v1

vis_util.tf = tf.compat.v1

In [6]:
import time

# ssd_mobilenet_v1_coco_2018_01_28 모델 사용시  
1. frozen_inference_graph 로 불러와 session 만들어서 직접 후 처리 (tf 1)
2. saved_model 불러와 직접 후 처리
3. tensorflow model research 에서 제공된 메소드 이용 처리  
(튜토리얼(제공메소드) 방식: https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb)

1. frozen_inference_graph 로 불러와 session 만들어서 직접 후 처리 (tf 1)

In [15]:
# 그래프 정의
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
with tf.compat.v1.gfile.GFile("ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb","rb") as f:
    graph = tf.compat.v1.GraphDef()
    graph.ParseFromString(f.read())

In [16]:
# 라벨 맵 불러오기 방법 2
# https://developers.google.com/protocol-buffers/docs/downloads 에서 다운받아 proto 파일들을 컴파일 해야 된다.
# https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt
# https://github.com/tensorflow/models/blob/master/research/object_detection/utils/label_map_util.py
from object_detection.utils.label_map_util import get_label_map_dict

tf.gfile = tf.io.gfile
get_label_map_dict.tf = tf.compat.v1

label_map = get_label_map_dict("ssd_mobilenet_v1_coco_2018_01_28/mscoco_complete_label_map.pbtxt", True)
label = dict()
for item in label_map.items():
    label[item[1]] = item[0]

In [17]:
# 세션 생성
with tf.compat.v1.Session() as sess:
    tf.compat.v1.import_graph_def(graph, name='')
    
    # 추론 이미지 생성
    capture = cv2.VideoCapture("files/bird.mp4") # https://www.youtube.com/watch?v=XdlIbNrki5o

    start = time.time()
    while True:
        ret, frame = capture.read()
        
        if (capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
            break
            
        input_img = cv2.resize(frame, (300,300))
        input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
        
        # 그래프 연산 수행
        num_detections, scores, classes, boxes = sess.run(
            fetches=[sess.graph.get_tensor_by_name("num_detections:0"),
                   sess.graph.get_tensor_by_name("detection_scores:0"),
                   sess.graph.get_tensor_by_name("detection_classes:0"),
                   sess.graph.get_tensor_by_name("detection_boxes:0")],
        feed_dict={"image_tensor:0": input_img.reshape(1, input_img.shape[0], input_img.shape[1],
                                                       input_img.shape[2])}
        )
        
        # 추론 결과 표시
        rows, cols, _ = frame.shape
        for i in range(int(num_detections[0])):
            class_id = int(classes[0][i])
            score = scores[0][i]
            box = boxes[0][i]
            
            if score > 0.7:
                x1 = int(box[1] * cols)
                y1 = int(box[0] * rows)
                x2 = int(box[3] * cols)
                y2 = int(box[2] * rows)
                

                cv2.putText(frame, label[class_id] + ":" + str(score), (x1, y1 - 5), cv2.FONT_HERSHEY_COMPLEX,
                            1.5, (0, 0, 255), 1)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 5)


        cv2.imshow("Object Detections", frame)
        if capture.get(cv2.CAP_PROP_POS_FRAMES) % 20 == 0:
            print("read time from {:d} ~ to {:d} : {:.3f}s".format(
                int(capture.get(cv2.CAP_PROP_POS_FRAMES)-20),int(capture.get(cv2.CAP_PROP_POS_FRAMES)),time.time()-start))
            start = time.time()
        if cv2.waitKey(33) == ord('q'): 
            cv2.destroyAllWindows()
            break

read time from 0 ~ to 20 : 4.371s
read time from 20 ~ to 40 : 1.661s
read time from 40 ~ to 60 : 1.656s
read time from 60 ~ to 80 : 1.657s
read time from 80 ~ to 100 : 1.654s
read time from 100 ~ to 120 : 1.649s


2. saved_model 불러와 직접 후 처리

In [8]:
def load_exist_model(model_name):
    model_dir = model_name
    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

In [9]:
model_name = "ssd_mobilenet_v1_coco_2018_01_28"
model = load_exist_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [10]:
# https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_label_map.pbtxt
# 라벨 맵 불러오기 방법 1
with open("ssd_mobilenet_v1_coco_2018_01_28/mscoco_label_map.pbtxt","rt") as f:
    pb_classes = f.read().rstrip('\n').split('\n')
    
    classes_label = dict()
    for i in range(0, len(pb_classes), 5):
        pb_classId = int(re.findall(r"\d+", pb_classes[i+2])[0])
        pattern = r"display_name: \"(.*?)\""
        pb_text = re.search(pattern, pb_classes[i+3])
        classes_label[pb_classId] = pb_text.group(1)

In [9]:
# # List of the strings that is used to add correct label for each box.
# PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
# category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [11]:
model.output_dtypes

{'detection_scores': tf.float32,
 'detection_classes': tf.float32,
 'num_detections': tf.float32,
 'detection_boxes': tf.float32}

In [12]:
model.output_shapes

{'detection_scores': TensorShape([None, 100]),
 'detection_classes': TensorShape([None, 100]),
 'num_detections': TensorShape([None]),
 'detection_boxes': TensorShape([None, 100, 4])}

In [13]:
#비디오에서 적용

capture = cv2.VideoCapture("files/bird.mp4") # https://www.youtube.com/watch?v=XdlIbNrki5o

start = time.time()
while True:
    
    ret, frame = capture.read()
    
    if (capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        break
        
    image = np.asarray(frame)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]

    output_dict = model(input_tensor)

    # 추론 결과 표시
    rows, cols, _ = image.shape
    for i in range(int(output_dict['num_detections'])):
        class_id = int(output_dict['detection_classes'][0][i])
        score = output_dict['detection_scores'][0][i].numpy()
        box = output_dict['detection_boxes'][0][i]

        if score > 0.7:
            x1 = int(box[1] * cols)
            y1 = int(box[0] * rows)
            x2 = int(box[3] * cols)
            y2 = int(box[2] * rows)


            cv2.putText(image, classes_label[class_id] + ":" + str(score), (x1, y1 - 5), cv2.FONT_HERSHEY_COMPLEX,
                        1.5, (0, 0, 255), 1)
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 5)

    cv2.imshow("Object Detections", image)
    if capture.get(cv2.CAP_PROP_POS_FRAMES) % 20 == 0:
            print("read time from {:d} ~ to {:d} : {:.3f}s".format(
                int(capture.get(cv2.CAP_PROP_POS_FRAMES)-20),int(capture.get(cv2.CAP_PROP_POS_FRAMES)),time.time()-start))
            start = time.time()
    if cv2.waitKey(33) == ord('q'): 
        cv2.destroyAllWindows()
        break

read time from 0 ~ to 20 : 6.435s
read time from 20 ~ to 40 : 1.742s
read time from 40 ~ to 60 : 1.717s
read time from 60 ~ to 80 : 1.725s
read time from 80 ~ to 100 : 1.753s
read time from 100 ~ to 120 : 1.701s
read time from 120 ~ to 140 : 1.696s
read time from 140 ~ to 160 : 1.656s
read time from 160 ~ to 180 : 1.738s
read time from 180 ~ to 200 : 1.746s
read time from 200 ~ to 220 : 1.732s
read time from 220 ~ to 240 : 1.728s
read time from 240 ~ to 260 : 1.730s
read time from 260 ~ to 280 : 1.730s
read time from 280 ~ to 300 : 1.741s
read time from 300 ~ to 320 : 1.830s
read time from 320 ~ to 340 : 1.743s
read time from 340 ~ to 360 : 2.342s
read time from 360 ~ to 380 : 2.540s
read time from 380 ~ to 400 : 2.532s
read time from 400 ~ to 420 : 2.496s
read time from 420 ~ to 440 : 2.661s
read time from 440 ~ to 460 : 2.096s
read time from 460 ~ to 480 : 2.565s
read time from 480 ~ to 500 : 1.762s
read time from 500 ~ to 520 : 1.747s
read time from 520 ~ to 540 : 2.493s
read time f

3. tensorflow model research 에서 제공된 메소드 이용 처리 

In [14]:
def load_exist_model(model_name):
    model_dir = model_name
    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

In [15]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    output_dict = model(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
     
    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                            output_dict['detection_masks'], output_dict['detection_boxes'],
                             image.shape[0], image.shape[1])            
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                                                             tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
        
    return output_dict

In [16]:
def show_inference_test2(model, image):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(image)
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks_reframed', None),
            use_normalized_coordinates=True,
            min_score_thresh = .7,
            line_thickness=8)

    return image_np

In [17]:
model_name = "ssd_mobilenet_v1_coco_2018_01_28"
model = load_exist_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [18]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [19]:
# 비디오에서 적용    
# 추론 이미지 생성
capture = cv2.VideoCapture("files/bird.mp4") # https://www.youtube.com/watch?v=XdlIbNrki5o
start = time.time()
while True:
    ret, frame = capture.read()
        
    if (capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        break
        
    frame = show_inference_test2(model, frame)

    cv2.imshow("Object Detections", frame)
    
    if capture.get(cv2.CAP_PROP_POS_FRAMES) % 20 == 0:
            print("read time from {:d} ~ to {:d} : {:.3f}s".format(
                int(capture.get(cv2.CAP_PROP_POS_FRAMES)-20),int(capture.get(cv2.CAP_PROP_POS_FRAMES)),time.time()-start))
            start = time.time()
            
    if cv2.waitKey(33) == ord('q'): 
        cv2.destroyAllWindows()
        break
    #     cv2.imshow("Object Detections", img)
    #     cv2.waitKey(0) 
    #     cv2.destroyAllWindows()

read time from 0 ~ to 20 : 4.973s
read time from 20 ~ to 40 : 1.932s
read time from 40 ~ to 60 : 1.931s
read time from 60 ~ to 80 : 1.932s
read time from 80 ~ to 100 : 1.928s
read time from 100 ~ to 120 : 1.986s
read time from 120 ~ to 140 : 1.985s
read time from 140 ~ to 160 : 1.982s
read time from 160 ~ to 180 : 1.959s
read time from 180 ~ to 200 : 1.952s
read time from 200 ~ to 220 : 1.990s
read time from 220 ~ to 240 : 1.982s
read time from 240 ~ to 260 : 1.974s
read time from 260 ~ to 280 : 1.992s
read time from 280 ~ to 300 : 1.995s
read time from 300 ~ to 320 : 2.076s


# mask_rcnn_inception_v2_coco_2018_01_28 모델 사용시  
1. frozen_inference_graph 로 불러와 session 만들어서 직접 후 처리 (tf 1)
2. saved_model 불러와 직접 후 처리
3. tensorflow model research 에서 제공된 메소드 이용 처리  
(튜토리얼(제공메소드) 방식: https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb)

1. frozen_inference_graph 로 불러와 session 만들어서 직접 후 처리 (tf 1)

반복문 seesion 안에서도 계속 seesion을 실행하기 때문에 너무 오래걸리는 것 같다. 이건 내가 하는 방법을 몰라서 그런듯..

In [7]:
import PIL.ImageColor as ImageColor

In [8]:
# https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py
STANDARD_COLORS = [
    'AliceBlue', 'Chartreuse', 'Aqua', 'Aquamarine', 'Azure', 'Beige', 'Bisque',
    'BlanchedAlmond', 'BlueViolet', 'BurlyWood', 'CadetBlue', 'AntiqueWhite',
    'Chocolate', 'Coral', 'CornflowerBlue', 'Cornsilk', 'Crimson', 'Cyan',
    'DarkCyan', 'DarkGoldenRod', 'DarkGrey', 'DarkKhaki', 'DarkOrange',
    'DarkOrchid', 'DarkSalmon', 'DarkSeaGreen', 'DarkTurquoise', 'DarkViolet',
    'DeepPink', 'DeepSkyBlue', 'DodgerBlue', 'FireBrick', 'FloralWhite',
    'ForestGreen', 'Fuchsia', 'Gainsboro', 'GhostWhite', 'Gold', 'GoldenRod',
    'Salmon', 'Tan', 'HoneyDew', 'HotPink', 'IndianRed', 'Ivory', 'Khaki',
    'Lavender', 'LavenderBlush', 'LawnGreen', 'LemonChiffon', 'LightBlue',
    'LightCoral', 'LightCyan', 'LightGoldenRodYellow', 'LightGray', 'LightGrey',
    'LightGreen', 'LightPink', 'LightSalmon', 'LightSeaGreen', 'LightSkyBlue',
    'LightSlateGray', 'LightSlateGrey', 'LightSteelBlue', 'LightYellow', 'Lime',
    'LimeGreen', 'Linen', 'Magenta', 'MediumAquaMarine', 'MediumOrchid',
    'MediumPurple', 'MediumSeaGreen', 'MediumSlateBlue', 'MediumSpringGreen',
    'MediumTurquoise', 'MediumVioletRed', 'MintCream', 'MistyRose', 'Moccasin',
    'NavajoWhite', 'OldLace', 'Olive', 'OliveDrab', 'Orange', 'OrangeRed',
    'Orchid', 'PaleGoldenRod', 'PaleGreen', 'PaleTurquoise', 'PaleVioletRed',
    'PapayaWhip', 'PeachPuff', 'Peru', 'Pink', 'Plum', 'PowderBlue', 'Purple',
    'Red', 'RosyBrown', 'RoyalBlue', 'SaddleBrown', 'Green', 'SandyBrown',
    'SeaGreen', 'SeaShell', 'Sienna', 'Silver', 'SkyBlue', 'SlateBlue',
    'SlateGray', 'SlateGrey', 'Snow', 'SpringGreen', 'SteelBlue', 'GreenYellow',
    'Teal', 'Thistle', 'Tomato', 'Turquoise', 'Violet', 'Wheat', 'White',
    'WhiteSmoke', 'Yellow', 'YellowGreen'
]

In [9]:
# 그래프 정의
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
with tf.compat.v1.gfile.GFile("mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb","rb") as f:
    graph = tf.compat.v1.GraphDef()
    graph.ParseFromString(f.read())

In [10]:
# https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_label_map.pbtxt
# 라벨 맵 불러오기 방법 1
with open("object_detection/data/mscoco_label_map.pbtxt","rt") as f:
    pb_classes = f.read().rstrip('\n').split('\n')
    
    classes_label = dict()
    for i in range(0, len(pb_classes), 5):
        pb_classId = int(re.findall(r"\d+", pb_classes[i+2])[0])
        pattern = r"display_name: \"(.*?)\""
        pb_text = re.search(pattern, pb_classes[i+3])
        classes_label[pb_classId] = pb_text.group(1)

In [11]:
# 세션 생성
with tf.compat.v1.Session() as sess:
    tf.compat.v1.import_graph_def(graph, name='')
    
    # 추론 이미지 생성
    capture = cv2.VideoCapture("files/bird.mp4") # https://www.youtube.com/watch?v=XdlIbNrki5o

    start = time.time()
    while True:
        ret, frame = capture.read()
        
        if (capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
            break
            
        input_img = cv2.resize(frame, (300,300))
        input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
        
        # 그래프 연산 수행
        num_detections, scores, classes, boxes, masks = sess.run(
            fetches=[sess.graph.get_tensor_by_name("num_detections:0"),
                    sess.graph.get_tensor_by_name("detection_scores:0"),
                    sess.graph.get_tensor_by_name("detection_classes:0"),
                    sess.graph.get_tensor_by_name("detection_boxes:0"),
                    sess.graph.get_tensor_by_name("detection_masks:0")],
        feed_dict={"image_tensor:0": input_img.reshape(1, input_img.shape[0], input_img.shape[1],
                                                        input_img.shape[2])}
        )
        
        # 추론 결과 표시
        rows, cols, _ = frame.shape
        for i in range(int(num_detections[0])):
            class_id = int(classes[0][i])
            score = scores[0][i]
            box = boxes[0][i]
            mask = masks[0][i]
            
            if score > 0.7:
                x1 = int(box[1] * cols)
                y1 = int(box[0] * rows)
                x2 = int(box[3] * cols)
                y2 = int(box[2] * rows)
                

                cv2.putText(frame, classes_label[class_id] + ":" + str(score), (x1, y1 - 5), cv2.FONT_HERSHEY_COMPLEX,
                            1.5, (0, 0, 255), 1)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 5)
                
#                 mask = tf.expand_dims(mask,axis=-1)
                mask = tf.compat.v1.expand_dims(mask,axis=-1)
#                 mask = sess.run(mask)
                mask = tf.compat.v1.image.resize(mask,(y2-y1,x2-x1))
#                 mask = sess.run(mask)
                mask_reframed = tf.compat.v1.cast(mask > 0.5, tf.uint8)
#                 mask_reframed = sess.run(mask_reframed)
                mask_reframed = tf.compat.v1.squeeze(mask_reframed)
                mask_reframed = sess.run(mask_reframed)
                # for display
        #         mask_reframed_d = mask_reframed * 255
        #         display(Image.fromarray(mask_reframed_d.numpy()))

                full_mask = np.zeros((rows, cols), dtype=np.uint8)
                full_mask[y1:y2,x1:x2] = mask_reframed
                # for display
        #         full_mask_d = full_mask * 255
        #         display(Image.fromarray(full_mask_d))

                image = np.asarray(frame)
                pil_image = Image.fromarray(image)
                color = STANDARD_COLORS[class_id%len(STANDARD_COLORS)]

                alpha = 0.4
                rgb = ImageColor.getrgb(color)

                solid_color = np.expand_dims(np.ones_like(full_mask), axis=2) * np.reshape(list(rgb), [1, 1, 3])
                pil_solid_color = Image.fromarray(np.uint8(solid_color)).convert('RGBA')
                pil_mask = Image.fromarray(np.uint8(255.0*alpha*full_mask)).convert('L')
                pil_image = Image.composite(pil_solid_color, pil_image, pil_mask)
                np.copyto(image, np.array(pil_image.convert('RGB')))


        cv2.imshow("Object Detections", frame)
        if capture.get(cv2.CAP_PROP_POS_FRAMES) % 20 == 0:
            print("read time from {:d} ~ to {:d} : {:.3f}s".format(
                int(capture.get(cv2.CAP_PROP_POS_FRAMES)-20),int(capture.get(cv2.CAP_PROP_POS_FRAMES)),time.time()-start))
            start = time.time()
        if cv2.waitKey(33) == ord('q'): 
            cv2.destroyAllWindows()
            break

read time from 0 ~ to 20 : 15.158s
read time from 20 ~ to 40 : 16.123s
read time from 40 ~ to 60 : 19.095s
read time from 60 ~ to 80 : 15.964s
read time from 80 ~ to 100 : 14.373s
read time from 100 ~ to 120 : 19.174s


2. saved_model 불러와 직접 후 처리

In [7]:
import PIL.ImageColor as ImageColor

In [8]:
def load_exist_model(model_name):
    model_dir = model_name
    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

In [9]:
# https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py
STANDARD_COLORS = [
    'AliceBlue', 'Chartreuse', 'Aqua', 'Aquamarine', 'Azure', 'Beige', 'Bisque',
    'BlanchedAlmond', 'BlueViolet', 'BurlyWood', 'CadetBlue', 'AntiqueWhite',
    'Chocolate', 'Coral', 'CornflowerBlue', 'Cornsilk', 'Crimson', 'Cyan',
    'DarkCyan', 'DarkGoldenRod', 'DarkGrey', 'DarkKhaki', 'DarkOrange',
    'DarkOrchid', 'DarkSalmon', 'DarkSeaGreen', 'DarkTurquoise', 'DarkViolet',
    'DeepPink', 'DeepSkyBlue', 'DodgerBlue', 'FireBrick', 'FloralWhite',
    'ForestGreen', 'Fuchsia', 'Gainsboro', 'GhostWhite', 'Gold', 'GoldenRod',
    'Salmon', 'Tan', 'HoneyDew', 'HotPink', 'IndianRed', 'Ivory', 'Khaki',
    'Lavender', 'LavenderBlush', 'LawnGreen', 'LemonChiffon', 'LightBlue',
    'LightCoral', 'LightCyan', 'LightGoldenRodYellow', 'LightGray', 'LightGrey',
    'LightGreen', 'LightPink', 'LightSalmon', 'LightSeaGreen', 'LightSkyBlue',
    'LightSlateGray', 'LightSlateGrey', 'LightSteelBlue', 'LightYellow', 'Lime',
    'LimeGreen', 'Linen', 'Magenta', 'MediumAquaMarine', 'MediumOrchid',
    'MediumPurple', 'MediumSeaGreen', 'MediumSlateBlue', 'MediumSpringGreen',
    'MediumTurquoise', 'MediumVioletRed', 'MintCream', 'MistyRose', 'Moccasin',
    'NavajoWhite', 'OldLace', 'Olive', 'OliveDrab', 'Orange', 'OrangeRed',
    'Orchid', 'PaleGoldenRod', 'PaleGreen', 'PaleTurquoise', 'PaleVioletRed',
    'PapayaWhip', 'PeachPuff', 'Peru', 'Pink', 'Plum', 'PowderBlue', 'Purple',
    'Red', 'RosyBrown', 'RoyalBlue', 'SaddleBrown', 'Green', 'SandyBrown',
    'SeaGreen', 'SeaShell', 'Sienna', 'Silver', 'SkyBlue', 'SlateBlue',
    'SlateGray', 'SlateGrey', 'Snow', 'SpringGreen', 'SteelBlue', 'GreenYellow',
    'Teal', 'Thistle', 'Tomato', 'Turquoise', 'Violet', 'Wheat', 'White',
    'WhiteSmoke', 'Yellow', 'YellowGreen'
]

In [10]:
# https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_label_map.pbtxt
# 라벨 맵 불러오기 방법 1
with open("object_detection/data/mscoco_label_map.pbtxt","rt") as f:
    pb_classes = f.read().rstrip('\n').split('\n')
    
    classes_label = dict()
    for i in range(0, len(pb_classes), 5):
        pb_classId = int(re.findall(r"\d+", pb_classes[i+2])[0])
        pattern = r"display_name: \"(.*?)\""
        pb_text = re.search(pattern, pb_classes[i+3])
        classes_label[pb_classId] = pb_text.group(1)

In [11]:
model_name = "mask_rcnn_inception_v2_coco_2018_01_28"
model = load_exist_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [12]:
# # List of the strings that is used to add correct label for each box.
# PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
# category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [12]:
model.output_dtypes

{'detection_scores': tf.float32,
 'detection_classes': tf.float32,
 'detection_masks': tf.float32,
 'num_detections': tf.float32,
 'detection_boxes': tf.float32}

In [13]:
model.output_shapes

{'detection_scores': TensorShape([None, 100]),
 'detection_classes': TensorShape([None, 100]),
 'detection_masks': TensorShape([None, None, None, None]),
 'num_detections': TensorShape([None]),
 'detection_boxes': TensorShape([None, 100, 4])}

In [14]:
#비디오에서 적용

capture = cv2.VideoCapture("files/bird.mp4") # https://www.youtube.com/watch?v=XdlIbNrki5o

start = time.time()
while True:
    
    ret, frame = capture.read()
    
    if (capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        break
        
    image = np.asarray(frame)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]

    output_dict = model(input_tensor)

    # 추론 결과 표시
    rows, cols, _ = image.shape
    for i in range(int(output_dict['num_detections'])):
        class_id = int(output_dict['detection_classes'][0][i])
        score = output_dict['detection_scores'][0][i].numpy()
        box = output_dict['detection_boxes'][0][i]
        mask = output_dict['detection_masks'][0][i]

        if score > 0.7:
            x1 = int(box[1] * cols)
            y1 = int(box[0] * rows)
            x2 = int(box[3] * cols)
            y2 = int(box[2] * rows)


            cv2.putText(image, classes_label[class_id] + ":" + str(score), (x1, y1 - 5), cv2.FONT_HERSHEY_COMPLEX,
                        1.5, (0, 0, 255), 1)
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 5)
            
            mask = tf.expand_dims(mask,axis=-1)        
            mask = tf.image.resize(mask,(y2-y1,x2-x1))
            mask_reframed = tf.cast(mask > 0.5, tf.uint8)
            mask_reframed = tf.squeeze(mask_reframed)
            # for display
    #         mask_reframed_d = mask_reframed * 255
    #         display(Image.fromarray(mask_reframed_d.numpy()))

            full_mask = np.zeros((rows, cols), dtype=np.uint8)
            full_mask[y1:y2,x1:x2] = mask_reframed
            # for display
    #         full_mask_d = full_mask * 255
    #         display(Image.fromarray(full_mask_d))

            pil_image = Image.fromarray(image)
            color = STANDARD_COLORS[class_id%len(STANDARD_COLORS)]

            alpha = 0.4
            rgb = ImageColor.getrgb(color)

            solid_color = np.expand_dims(np.ones_like(full_mask), axis=2) * np.reshape(list(rgb), [1, 1, 3])
            pil_solid_color = Image.fromarray(np.uint8(solid_color)).convert('RGBA')
            pil_mask = Image.fromarray(np.uint8(255.0*alpha*full_mask)).convert('L')
            pil_image = Image.composite(pil_solid_color, pil_image, pil_mask)
            np.copyto(image, np.array(pil_image.convert('RGB')))
            

    cv2.imshow("Object Detections", image)
    if capture.get(cv2.CAP_PROP_POS_FRAMES) % 20 == 0:
            print("read time from {:d} ~ to {:d} : {:.3f}s".format(
                int(capture.get(cv2.CAP_PROP_POS_FRAMES)-20),int(capture.get(cv2.CAP_PROP_POS_FRAMES)),time.time()-start))
            start = time.time()
    if cv2.waitKey(33) == ord('q'): 
        cv2.destroyAllWindows()
        break

read time from 0 ~ to 20 : 15.723s
read time from 20 ~ to 40 : 5.793s
read time from 40 ~ to 60 : 5.274s
read time from 60 ~ to 80 : 5.244s
read time from 80 ~ to 100 : 5.664s
read time from 100 ~ to 120 : 5.354s
read time from 120 ~ to 140 : 5.470s
read time from 140 ~ to 160 : 5.572s
read time from 160 ~ to 180 : 5.824s
read time from 180 ~ to 200 : 5.663s
read time from 200 ~ to 220 : 5.621s
read time from 220 ~ to 240 : 5.492s
read time from 240 ~ to 260 : 5.390s
read time from 260 ~ to 280 : 5.528s
read time from 280 ~ to 300 : 5.171s
read time from 300 ~ to 320 : 5.563s


3. tensorflow model research 에서 제공된 메소드 이용 처리  

In [16]:
def run_inference_for_single_image_test(output_dict, image):
    image = np.asarray(image)
    
    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    
    output_dict = {key:value[0, :num_detections].numpy() 
                                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections
    
    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    
    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                            output_dict['detection_masks'], output_dict['detection_boxes'],
                             image.shape[0], image.shape[1])            
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                                                             tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
        
    return output_dict

In [17]:
def show_inference_test(output_dict, image):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(image)
    # Actual detection.
    output_dict = run_inference_for_single_image_test(output_dict, image_np)
    
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks_reframed', None),
            use_normalized_coordinates=True,
            min_score_thresh = .7,
            line_thickness=8)
    

#     display(Image.fromarray(image_np))
    
    return image_np

In [19]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [18]:
def load_exist_model(model_name):
    model_dir = model_name
    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

In [20]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    output_dict = model(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
     
    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                            output_dict['detection_masks'], output_dict['detection_boxes'],
                             image.shape[0], image.shape[1])            
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                                                             tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
        
    return output_dict

In [21]:
def show_inference_test2(model, image):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(image)
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks_reframed', None),
            use_normalized_coordinates=True,
            min_score_thresh = .7,
            line_thickness=8)

    return image_np

In [22]:
model_name = "mask_rcnn_inception_v2_coco_2018_01_28m"
masking_model = load_exist_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [24]:
# 세션 말고 정식 방법을 써보자.
# 진짜 훨씬 훨씬 빠르다. 진작 이 방법을 쓸걸 그랬다.

# 비디오에서 적용    
# 추론 이미지 생성
capture = cv2.VideoCapture("files/bird.mp4") # https://www.youtube.com/watch?v=XdlIbNrki5o

start = time.time()
while True:
    ret, frame = capture.read()
        
    if (capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        break
        
    frame = show_inference_test2(masking_model, frame)

    cv2.imshow("Object Detections", frame)
    
    if capture.get(cv2.CAP_PROP_POS_FRAMES) % 20 == 0:
            print("read time from {:d} ~ to {:d} : {:.3f}s".format(
                int(capture.get(cv2.CAP_PROP_POS_FRAMES)-20),int(capture.get(cv2.CAP_PROP_POS_FRAMES)),time.time()-start))
            start = time.time()
    if cv2.waitKey(33) == ord('q'): 
        cv2.destroyAllWindows()
        break
    #     cv2.imshow("Object Detections", img)
    #     cv2.waitKey(0) 
    #     cv2.destroyAllWindows()

read time from 0 ~ to 20 : 5.582s
read time from 20 ~ to 40 : 5.929s
read time from 40 ~ to 60 : 5.536s
read time from 60 ~ to 80 : 5.477s
read time from 80 ~ to 100 : 5.944s
read time from 100 ~ to 120 : 5.479s
read time from 120 ~ to 140 : 5.573s
read time from 140 ~ to 160 : 5.673s
read time from 160 ~ to 180 : 5.754s
